This notebook shows example of using "VERBOSE" parameter in TPOT regression model.

## <H2> Set up the development environment</H2>

In [ ]:
##After executing this code, automatically your session will be restarted witha message saying that "session crashed for unknown reason".
%%capture
! pip install DXC-Industrialized-AI-Starter
import os
os.kill(os.getpid(), 9)

From DXC-Industrialized-AI-Starter library import dxc-ai package to use it for further work.


In [ ]:
%%capture
from dxc import ai

## <H2> Access the raw data </H2>




In [ ]:
# TODO: Access raw data.
##reads json from a url and flattens it into a dataframe
data0 = ai.read_data_frame_from_remote_json("https://data.cincinnati-oh.gov/resource/ucjy-ykv4.json")
data0.head()

,request_id,request_status,funding_source,request_type,department_name,fiscal_year,replacement_body_style,equipment_class,equip_id_to_replace,replacement_make,replacement_model,date_fleet_doc_entered,fleet_document_type,purchasing_bid_date,est_unit_cost,date_bid_closed,date_po_awarded,actual_unit_cost,procurement_plan
0,463,Completed,CAP,Fleet Procurement Plan,Fire,2019,SEDAN,PC-GS,06193,CHEVROLET,MALIBU,2019-04-09T00:00:00.000,RQS,2019-07-05T00:00:00.000,19246,2019-07-05T00:00:00.000,2019-07-10T00:00:00.000,17507.14,Passenger Cars
1,674,Completed,Auction Funds,Supplemental Request,DPS - Fleet Services Division,2019,HYDRANT PUMP,NaN,NaN,MILWAUKEE,M18 LI-ION,2019-11-05T00:00:00.000,DO,NaN,400,NaN,NaN,355.55,Specialized Small
2,688,Denied,Operating Funds,Supplemental Request,DPS - Neighborhood Operations Division,2020,NaN,GROUNDS,NEW,NaN,NaN,NaN,DO,NaN,4631.8,NaN,NaN,NaN,NaN
3,818,Completed,Other,Supplemental Request,Police,2017,SUV,PC-GS,TBD,Chevrolet,Traverse,2020-06-01T00:00:00.000,DO,NaN,25799.5,NaN,NaN,25799.5,Passenger Cars
4,248,Completed,CIP,Fleet Procurement Plan,Recreation,2017,HEDGE TRIMMER,SPECIALIZED SMALL,46500,STIHL,"HS82T(24"")",2017-08-01T00:00:00.000,DO,NaN,250,NaN,2017-08-07T00:00:00.000,384.96,Horticulture


### <H2> Define data fields </H2>
Mention the <code>text_fileds</code>,<code>date_fields</code>,<code>numeric_fields</code> and <code>categorical_fields</code> as per you data set. Below are example only.

In [ ]:
# TODO: define the data fields
text_fields = []
date_fields = ['date_fleet_doc_entered', 'purchasing_bid_date', 'date_bid_closed', 'date_po_awarded']
numeric_fields = ['est_unit_cost', 'actual_unit_cost']
categorical_fields = ['request_id','request_status', 'funding_source', 'request_type', 'department_name', 
                      'fiscal_year', 'replacement_body_style','equipment_class','equip_id_to_replace',
                      'replacement_make', 'replacement_model','fleet_document_type', 'procurement_plan']

### <H2> Clean the raw data </H2>
Execute <code>raw_data</code> so that it accesses your raw data and returns it as a Pandas dataframe. Any preprocessing of the raw data should be done here. 

In [ ]:
#clean the data
impute = True
data1 = ai.clean_dataframe(data0, impute, text_fields, date_fields, numeric_fields, categorical_fields)

#display excerpts of the raw data
data1.head()

,request_id,request_status,funding_source,request_type,department_name,fiscal_year,replacement_body_style,equipment_class,equip_id_to_replace,replacement_make,replacement_model,date_fleet_doc_entered,fleet_document_type,purchasing_bid_date,est_unit_cost,date_bid_closed,date_po_awarded,actual_unit_cost,procurement_plan
0,463,Completed,CAP,Fleet Procurement Plan,Fire,2019,SEDAN,PC-GS,06193,CHEVROLET,MALIBU,2019-04-09T00:00:00+00:00,RQS,2019-07-05T00:00:00+00:00,19246.0,2019-07-05T00:00:00+00:00,2019-07-10T00:00:00+00:00,17507.14,Passenger Cars
1,674,Completed,Auction Funds,Supplemental Request,DPS - Fleet Services Division,2019,HYDRANT PUMP,PC-P,N/A,MILWAUKEE,M18 LI-ION,2019-11-05T00:00:00+00:00,DO,2019-08-13T00:00:00+00:00,400.0,2019-08-16T00:00:00+00:00,2019-10-31T00:00:00+00:00,355.55,Specialized Small
2,688,Denied,Operating Funds,Supplemental Request,DPS - Neighborhood Operations Division,2020,Cargo Van,GROUNDS,NEW,TORO,LEAF,2019-11-05T00:00:00+00:00,DO,2019-02-27T00:00:00+00:00,4631.8,2019-03-20T00:00:00+00:00,2018-06-21T00:00:00+00:00,4785.25,Passenger Cars
3,818,Completed,Other,Supplemental Request,Police,2017,SUV,PC-GS,TBD,Chevrolet,Traverse,2020-06-01T00:00:00+00:00,DO,2020-04-29T00:00:00+00:00,25799.5,2020-06-19T00:00:00+00:00,2020-06-12T00:00:00+00:00,25799.50,Passenger Cars
4,248,Completed,CIP,Fleet Procurement Plan,Recreation,2017,HEDGE TRIMMER,SPECIALIZED SMALL,46500,STIHL,"HS82T(24"")",2017-08-01T00:00:00+00:00,DO,2018-03-01T00:00:00+00:00,250.0,2018-03-16T00:00:00+00:00,2017-08-07T00:00:00+00:00,384.96,Horticulture


## <H1>Build a data pipeline</H1>

In [ ]:
# TODO: specify the details of the data layer
data_layer = {
    "connection_string": "<your connection_string>",
    "collection_name": "<your collection_name>",
    "database_name": "<your database_name>",
    "data_source":"<Source of your datset>",
    "cleaner":"<whether applied cleaner yes/no >"
}

Inject data into MongoDB

In [ ]:
data2 = ai.write_raw_data(data_layer, data1, date_fields)

In [ ]:
# TODO: define the code needed to refine the raw data
def data_pipeline():

  pipe = [
          {
              '$group':{
                  '_id': {
                      "funding_source":"$funding_source",
                      "request_type":"$request_type",
                      "department_name":"$department_name",
                      "replacement_body_style":"$replacement_body_style",
                      "equipment_class":"$equipment_class",
                      "replacement_make":"$replacement_make",
                      "replacement_model":"$replacement_model",
                      "procurement_plan":"$procurement_plan"
                      },
                  "avg_est_unit_cost":{"$avg":"$est_unit_cost"},
                  "avg_est_unit_cost_error":{"$avg":{ "$subtract": [ "$est_unit_cost", "$actual_unit_cost" ] }}
              }
          }
  ]

  return pipe

df = ai.access_data_from_pipeline(data2, data_pipeline())
df.head()

pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,avg_est_unit_cost,avg_est_unit_cost_error,_id.funding_source,_id.request_type,_id.department_name,_id.replacement_body_style,_id.equipment_class,_id.replacement_make,_id.replacement_model,_id.procurement_plan
0,37200.0,2017.096154,CAP,Fleet Procurement Plan,Police,SUV,PC-P,FORD,INTERCEPTOR,Emergency Response Vehicle
1,379.0,-31.290000,Auction Funds,Supplemental Request,Fire,HYDRANT PUMP,SPECIALIZED SMALL,milwaukee,m18,Specialized Small
2,29508.0,1619.300000,CIP,Fleet Procurement Plan,Risk Mgmt,SEDAN,PC-GS,NISSAN,LEAF,Passenger Cars
3,9000.0,2380.610000,CIP,Fleet Procurement Plan,Recreation,GATOR,GROUNDS,JOHN DEERE,GATOR 4X2,Horticulture
4,28000.0,-536.400000,CAP,Fleet Procurement Plan,Fire,SEDAN,ARV - Administrative Response Vehicles,DODGE,CHARGER,Emergency Response Vehicle


## <H1>Run an experiment</H1>

**TPOT Classification/Regression:**
Machine learning is  typically a very time-consuming and knowledge-intensive part of a data science problem. Auto-ml is not designed to replace the data scientist, but rather free her to work on more important aspects of the complete problem, such as acquiring data and interpreting the model results.[TPOT](https://epistasislab.github.io/tpot) is a Python library developed for automatic machine learning feature preprocessing, model selection, and hyperparameter tuning. AI Starter has integrated TPOT as one of its Auto Ml libraries. 

Please refer the below parameter description for best utilization of Tpot classification and regression methods with AI Starter.<br> 

**<code>verbose: True/False </code>** True - Prints more information and provide a progress bar; False - Prints nothing. <br>



**Parameter Verbose = False**

In [ ]:
# TODO: design and run an experiment
experiment_design = {
    #model options include ['tpot_regression()', 'tpot_classification()','timeseries()']
    "model": ai.tpot_regression(),
    "labels": df.avg_est_unit_cost_error,
    "data": df,
    #Tell the model which column is 'output'
    #Also note columns that aren't purely numerical
    #Examples include ['nlp', 'date', 'categorical', 'ignore']
    "meta_data": {
      "avg_est_unit_cost_error": "output",
      "_id.funding_source": "categorical",
      "_id.department_name": "categorical",
      "_id.replacement_body_style": "categorical",
      "_id.replacement_make": "categorical",
      "_id.replacement_model": "categorical",
      "_id.procurement_plan": "categorical"
  }
}

#config_dict = {'sklearn.ensemble.GradientBoostingRegressor':{}}

trained_model = ai.run_experiment(experiment_design, verbose = False, max_time_mins = 5, max_eval_time_mins = 0.04, config_dict = None, warm_start = False, export_pipeline = True, scoring = None)


r2 Score: -1.2041278860252143

Negative mean square error: -447363105.0795051

Negative root mean square error: -21150.959909174457

explained_variance: -1.1906717153409012

Negative_mean_absolute_error: -10680.754017568976

Negative_median_absolute_error: -3986.6064868209114



**Parameter Verbose = True**

In [ ]:
# TODO: design and run an experiment
experiment_design = {
    #model options include ['tpot_regression()', 'tpot_classification()','timeseries()']
    "model": ai.tpot_regression(),
    "labels": df.avg_est_unit_cost_error,
    "data": df,
    #Tell the model which column is 'output'
    #Also note columns that aren't purely numerical
    #Examples include ['nlp', 'date', 'categorical', 'ignore']
    "meta_data": {
      "avg_est_unit_cost_error": "output",
      "_id.funding_source": "categorical",
      "_id.department_name": "categorical",
      "_id.replacement_body_style": "categorical",
      "_id.replacement_make": "categorical",
      "_id.replacement_model": "categorical",
      "_id.procurement_plan": "categorical"
  }
}

#config_dict = {'sklearn.ensemble.GradientBoostingRegressor':{}}

trained_model = ai.run_experiment(experiment_design, verbose = True, max_time_mins = 5, max_eval_time_mins = 0.04, config_dict = None, warm_start = False, export_pipeline = True, scoring = None)


Generation 1 - Current best internal CV score: -1008610727.3658552

Generation 2 - Current best internal CV score: -997656513.3636204

Generation 3 - Current best internal CV score: -994217578.9332184

5.00 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: AdaBoostRegressor(SelectFwe(input_matrix, alpha=0.026000000000000002), learning_rate=1.0, loss=linear, n_estimators=100)

r2 Score: -1.324180239913305

Negative mean square error: -471729655.7447352

Negative root mean square error: -21719.3382897531

explained_variance: -1.3212160143461031

Negative_mean_absolute_error: -10736.584364654045

Negative_median_absolute_error: -4158.136265345765

